Run the following in your Terminal to get the gene names

curl -o ../hgnc_genes.tsv 'https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=gd_prev_sym&col=gd_aliases&status=Approved&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit'

In [8]:
from collections import defaultdict
import csv
class GeneValidator:
    def __init__(self, file_path):
        self.gene_symbol_set = set()
        self.alias_map = defaultdict(str)

        with open(file_path, 'r') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)  # Skip header row
            for cells in reader:
                if cells[0]:
                    gene = cells[0].upper()
                    self.gene_symbol_set.add(gene)

                    if len(cells) > 1 and cells[1]:
                        previous_symbols = cells[1].split(", ")
                        for symbol in previous_symbols:
                            self.alias_map[symbol.upper()] = gene

                    if len(cells) > 2 and cells[2]:
                        alias_symbols = cells[2].split(", ")
                        for alias in alias_symbols:
                            self.alias_map[alias.upper()] = gene

    def validate_human_genes(self, genes):
        official_genes = set()
        invalid_genes = set()
        updated_genes = {}

        for raw_term in genes:
            # print(f'validate Hugo symbol for {raw_term}')
            term = raw_term.upper()
            if term in self.gene_symbol_set:
                official_genes.add(term)
            elif term in self.alias_map:
                official_gene = self.alias_map[term]
                official_genes.add(official_gene)
                updated_genes[term] = official_gene
                invalid_genes.add(term)
            else:
                invalid_genes.add(term)

        return {
            'official_genes': official_genes,
            'invalid': invalid_genes,
            'updated_genes': updated_genes
        }


In [9]:
# Usage example:
file_path = "./hgnc_genes.tsv"

text = '''
	The significant upregulation of OAS family proteins, including OAS1 (2.309110681 at 24h, 2.327680098 at 48h), OAS2 (3.580865407 at 48h), and OAS3 (3.175544218 at 48h), along with OASL (2.916732074 at 48h), suggests a coordinated activation of the OAS-RNase L pathway in response to Dengue virus infection. This pathway is known to play a crucial role in the innate immune response against viral infections. The 2'-5'-oligoadenylate synthetases (OAS) are interferon-induced enzymes that, when activated by viral double-stranded RNA, produce 2'-5'-oligoadenylates. These oligoadenylates activate RNase L, which degrades viral and cellular RNAs, inhibiting protein synthesis and viral replication. The observed upregulation of multiple OAS family members indicates a robust activation of this antiviral mechanism. Interestingly, the data also shows a significant increase in IFIT family proteins, particularly IFIT1 (3.970117068 at 24h, 6.283329563 at 48h), IFIT2 (3.884688719 at 24h, 6.643829723 at 48h), and IFIT3 (3.494644087 at 24h, 5.69825699 at 48h). These proteins are known to recognize and sequester viral RNA, preventing its translation and replication. We hypothesize that the OAS-RNase L pathway and the IFIT family proteins work synergistically to create a multi-layered defense against Dengue virus. The OAS proteins may be primarily responsible for degrading viral RNA, while the IFIT proteins sequester any remaining viral RNA and prevent its translation. This coordinated response could explain the cell's ability to mount a strong antiviral state. To validate this hypothesis, siRNA knockdown experiments targeting OAS1, OAS2, OAS3, and IFIT1-3 could be performed in Dengue virus-infected cells. The effect on viral replication and cellular RNA degradation could be assessed through qRT-PCR and RNA sequencing. Additionally, overexpression of these proteins in susceptible cell lines could provide insights into their protective effects against Dengue virus infection. This hypothesis not only explains the observed upregulation of multiple antiviral proteins but also suggests potential targets for enhancing the cellular defense against Dengue virus.
'''

total_genes_set = text.split(' ')
validator = GeneValidator(file_path)
result = validator.validate_human_genes(total_genes_set)
# print(result)

updated_gene_symbols = list(result['official_genes'])
invalid_gene_symbols = list(result['invalid'])
updated_genes_mapping = result['updated_genes']

print(result['official_genes'])

{'official_genes': {'IFIT1', 'OAS2', 'WWOX', 'CELP', 'OAS1', 'OASL', 'SPARC', 'CD44', 'IFIT3', 'SPI1', 'OAS3', 'IFIT2', 'NR4A2'}, 'invalid': {'DENGUE', 'STRONG', 'REMAINING', 'ANTIVIRAL', 'PROTEINS', 'INCREASE', 'OF', 'VIRUS', 'SUGGESTS', 'AND', 'IN', 'PROTEINS,', 'ADDITIONALLY,', 'SHOWS', '(3.580865407', 'CRUCIAL', 'PROTEIN', 'INDICATES', 'INFECTION.', 'THEIR', 'VALIDATE', 'RNA,', 'TRANSLATION', 'KNOCKDOWN', "2'-5'-OLIGOADENYLATES.", 'SEQUENCING.', 'COULD', '(OAS)', 'DEGRADING', 'ACTIVATED', 'AT', 'INTO', 'IFIT1-3', '(2.916732074', 'EFFECT', 'WHEN', '(3.175544218', 'WORK', 'BE', 'CELL', '2.327680098', 'FAMILY', 'ITS', 'SIGNIFICANT', 'MECHANISM.', 'ALSO', 'SYNTHETASES', 'THROUGH', 'BUT', 'DATA', 'THAT', 'HYPOTHESIZE', 'STATE.', 'INNATE', 'BY', 'OAS3,', 'RNASE', 'L,', 'DEFENSE', 'KNOWN', 'IMMUNE', 'VIRAL', 'TO', "2'-5'-OLIGOADENYLATE", 'OLIGOADENYLATES', "CELL'S", 'NOT', 'MULTI-LAYERED', 'MAY', 'THAT,', 'ANY', 'POTENTIAL', 'CELLS.', '(3.494644087', 'INSIGHTS', 'REPLICATION.', 'PATHWAY',